In [1]:
%%time
import json
from sParser import Word, Parse_result, stanford_simplify, KB

CPU times: user 12.3 s, sys: 3.11 s, total: 15.4 s
Wall time: 15.2 s


In [18]:
df = pd.read_csv('short_data.csv',encoding='utf8')
df2 = df.loc[:,('id','phrase','explanation')]
df3= df2.dropna().reset_index(drop=True)

In [19]:
f_stop = open('哈工大停用词表.txt',encoding='utf8')
f_stop_text = f_stop.read( )
f_stop.close( )
f_stop_seg_list=f_stop_text.split('\n')

len(f_stop_seg_list)

768

In [20]:
def jiebaCleanText(text):
    #cut = jieba.cut(text)
    #cut_list=list(cut)
    #
    #text_word_list=[]
    #for myword in cut_list:
    #    #if not(myword.strip() in f_stop_seg_list) and len(myword.strip())>1:
    #    if not(myword.strip() in f_stop_seg_list):    
    #        text_word_list.append(myword)
    #return text_word_list
    key_word_list = ['形容','比喻','指','表示','常']
    try:
        psg_list=[[x.word,x.flag] for x in psg.cut(text)]
    except ValueError:
        return np.nan

    psg_key_list =[]
    tmp_index1=-1
    for i in range(len(psg_list)):
        if psg_list[i][0] in key_word_list:
            tmp_index1=i
        if psg_list[i][0]=='。' and tmp_index1 !=-1:
            psg_key_list+=psg_list[tmp_index1+1:i]
            tmp_index1=-1
    if len(psg_key_list) ==0:
        psg_key_list = psg_list

    final_list=[]    
    no_label=0
    for x in psg_key_list:
        if not('不' in x[0]) and no_label ==0 and (x[1] == 'd' or (x[0].strip() in f_stop_seg_list) or (len(x[0].strip()) ==1 and x[1] !='a')):
            continue
        elif x[0] == '不':
            no_label = 1
        elif no_label == 1:
            final_list.append('不'+x[0])
            no_label=0
        else:
            final_list.append(x[0])

    return final_list


df3['exp_word'] = df3.apply(lambda row:jiebaCleanText(row['explanation']), axis=1)


In [21]:
df3.to_csv('exp_word.csv')

In [22]:
from gensim import corpora, models, similarities
import gensim

docs = df3['exp_word'] 
dictionary = corpora.Dictionary(docs)
corpus = [dictionary.doc2bow(doc) for doc in docs]


In [24]:
for number_topics in [100,500,1000,1500]:
    for pass_number in [5,10,15,20]:
        
        lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=number_topics,passes=pass_number)
        
        #根据阈值找出对应的标签
        topic_cut_off=0.15
        word_cut_off=0.1

        result=[]
        for i in range(len(df3)):
            match={}
            match['phrase']=df3['phrase'][i]
            match['explanation']=docs[i]
            i_lda = lda[dictionary.doc2bow(docs[i])]
            for topic,p in i_lda:
                if p>topic_cut_off:
                    for word,p_word in lda.show_topic(topic,topn=10):
                        if p_word >word_cut_off:
                            match['label']=word
                            match['p_topic']=p
                            match['p_word']=p_word
                            result.append(match)
        df5=pd.DataFrame(result)
        
        file_name='result_'+str(number_topics) + '_' +str(pass_number) +'.csv'
        df5.to_csv(file_name)

C:\Users\guoyu9\AppData\Local\Continuum\anaconda3\envs\tf\lib\site-packages\gensim\models\ldamodel.py:775: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)
C:\Users\guoyu9\AppData\Local\Continuum\anaconda3\envs\tf\lib\site-packages\gensim\models\ldamodel.py:582: RuntimeWarning: overflow encountered in exp2
  perwordbound, np.exp2(-perwordbound), len(chunk), corpus_words
